In [1]:
# data augmentation. This code performs kfold=5
# ta505 CSM data
# my data from a mat file: first 5 rows = VN lang positive. next 
#13 rows = language negative
# DI features = 104, 
# indeg features = 104
# outdeg features = 104
# Kcores features = 104
# power features = 104
# edge betweenness features = 104
import scipy.io
import numpy as np
from matplotlib.pyplot import imshow

npos=5
nneg = 13
X = scipy.io.loadmat('X_ta505.mat',struct_as_record=False)
X_total = X['X']
y=np.array([1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
y=y.T

#DI features
Xtemp = X_total[:,0:104] #original DI matrix
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
XDI = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented DI matrix
print(XDI.shape)

#indeg features
Xtemp = X_total[:,104:208]
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
X_indeg = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented matrix

#outdeg features
Xtemp = X_total[:,208:312]
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
X_outdeg = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented matrix

#indeg and outdeg features combined
X_in_out_deg = np.concatenate((X_indeg,X_outdeg),axis=1)

#Kcores features
Xtemp = X_total[:,312:416]
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
X_kcores = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented matrix

#power features
Xtemp = X_total[:,416:520]
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
X_power = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented matrix

#edge features
Xtemp = X_total[:,520:624]
X1 = Xtemp[0:npos,0:52]
X2 = Xtemp[0:npos,52:104]
Xnew = np.concatenate((X2,X1),axis=1) #interchange the node order
X1n = Xtemp[npos:npos+nneg,0:52]
X2n = Xtemp[npos:npos+nneg,52:104]
Xnew_n = np.concatenate((X2n,X1n),axis=1) #interchange the node order
X_edge = np.concatenate((Xnew,Xtemp,Xnew_n),axis=0) #augmented matrix

#kcores and edge features combined
X_kcores_edge = np.concatenate((X_kcores,X_edge),axis=1)

#kcores and power combined
X_kcores_power = np.concatenate((X_kcores,X_power),axis=1)

#Edge and power combined
X_edge_power = np.concatenate((X_edge,X_power),axis=1)

#kcores+ edge + power combined
X_kcores_edge_power = np.concatenate((X_kcores_edge,X_power),axis=1)

#In + power combined
X_In_power = np.concatenate((X_indeg,X_power),axis=1)

#Out + power combined
X_Out_power = np.concatenate((X_outdeg,X_power),axis=1)

#InOut + power combined
X_InOut_power = np.concatenate((X_in_out_deg,X_power),axis=1)

# Inout+Kcores+power
X_InOut_kcores_power = np.concatenate((X_in_out_deg,X_kcores_power),axis=1)

nElements,pFeatures = X_kcores_edge_power.shape

print(X_kcores_edge_power.shape)
#imshow(X_power)
#plt.show()

(36, 104)
(36, 312)


In [2]:
# original Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for classification for CSM data by Sudha Yellapantula
# License: BSD 3 clause
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold # import KFold
from sklearn import linear_model
from sklearn.metrics import balanced_accuracy_score, average_precision_score, precision_score, recall_score,confusion_matrix

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
nClassifiers = 4
nDataSets = 11
maxRep=100
n_splits=5

prec = np.zeros((nClassifiers,nDataSets,n_splits,maxRep))
recall = np.zeros((nClassifiers,nDataSets,n_splits,maxRep))
bal_acc = np.zeros((nClassifiers,nDataSets,n_splits,maxRep))
tnr = np.zeros((nClassifiers,nDataSets,n_splits,maxRep))
for indRep in range(0,maxRep):

    classifiers = [
       KNeighborsClassifier(3),
        LinearSVC(C=0.001, class_weight='balanced', dual=True, fit_intercept=False,
         intercept_scaling=1, loss='squared_hinge', max_iter=10000,
         multi_class='ovr', penalty='l2', random_state=None, tol=0.0000001,
         verbose=0),
        GaussianProcessClassifier(1.0 * RBF(1.0),random_state=None),
        DecisionTreeClassifier(max_depth=20,random_state=None)
    ]

    Power_features = (X_power,y)
    Indeg_features = (X_indeg,y)
    Outdeg_features = (X_outdeg,y)
    In_out_deg_features = (X_in_out_deg,y)
    Kcores_features = (X_kcores,y)
    Kcores_power_features = (X_kcores_power,y)
    In_power_features = (X_In_power,y)
    Out_power_features = (X_Out_power,y)
    InOut_power_features = (X_InOut_power,y)
    Kcores_In_Out_power_features = (X_InOut_kcores_power,y)

    datasets = [Power_features,
                Indeg_features,
                Outdeg_features,
                In_out_deg_features,
                Kcores_features,
                Kcores_power_features,
                In_power_features,
                Out_power_features,
                InOut_power_features,
                Kcores_In_Out_power_features
                ]

    #predictions = np.zeros((len(classifiers),len(datasets), nElements))
    kf = StratifiedKFold(n_splits=5,random_state=None, shuffle=True)
    ic,ids,ie=0,0,0 #counter index for classifier, dataset and element

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        # iterate over datasets
        ids=0
        for ds_cnt, ds in enumerate(datasets):
            # preprocess dataset, split into training and test part
            X, y = ds
            #X,y = SubSampler(X,y)
            X = StandardScaler().fit_transform(X)

            ie=0
            for train_index, test_index in kf.split(X,y):
                X_train, X_test = X[train_index,:], X[test_index,:]
                y_train, y_test = y[train_index], y[test_index]

                clf.fit(X_train, y_train)
                score = clf.score(X_test, y_test)
                #predictions[ic,ids,ie,indRep]=score
                y_pred = clf.predict(X_test)
                CM = confusion_matrix(y_test, y_pred)
                TN = CM[0][0]
                FN = CM[1][0]
                TP = CM[1][1]
                FP = CM[0][1]
                bal_acc[ic,ids,ie,indRep]= balanced_accuracy_score(y_test, y_pred)
                prec[ic,ids,ie,indRep]= precision_score(y_test, y_pred)
                recall[ic,ids,ie,indRep]= recall_score(y_test, y_pred)
                tnr[ic,ids,ie,indRep]= np.true_divide(TN, (TN + FP))
                ie +=1
            ids +=1
        ic +=1



/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

...

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sudhayellapantula/opt/anaconda3/envs/langCSM/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

In [3]:
#for each rep, calculate these parameters
np.set_printoptions(precision=3)
prec_final = np.zeros((len(datasets),len(classifiers)))
tnrate = np.zeros((len(datasets),len(classifiers)))
rec = np.zeros((len(datasets),len(classifiers)))
acc = np.zeros((len(datasets),len(classifiers)))


ste_prec = np.zeros((len(datasets),len(classifiers)))
ste_tnrate = np.zeros((len(datasets),len(classifiers)))
ste_rec = np.zeros((len(datasets),len(classifiers)))
ste_acc = np.zeros((len(datasets),len(classifiers)))

for i in range(0,len(datasets)):
    for j in range(0,len(classifiers)):
        tnrate[i,j] = np.mean(tnr[j,i,:,:])
        prec_final[i,j] = np.mean(prec[j,i,:,:])
        rec[i,j] = np.mean(recall[j,i,:,:])
        acc[i,j] = np.mean(bal_acc[j,i,:,:])
        
for i in range(0,len(datasets)):
    for j in range(0,len(classifiers)):     
        ste_tnrate[i,j] = 1.96*np.true_divide(np.std(tnr[j,i,:,:]),np.sqrt(maxRep*n_splits))
        ste_prec[i,j] = 1.96*np.true_divide(np.std(prec[j,i,:,:]),np.sqrt(maxRep*n_splits))
        ste_rec[i,j] = 1.96*np.true_divide(np.std(recall[j,i,:,:]),np.sqrt(maxRep*n_splits))
        ste_acc[i,j] =1.96*np.true_divide(np.std(bal_acc[j,i,:,:]),np.sqrt(maxRep*n_splits))

    

print("mean accuracy\n",acc)
print("ste of accuracy\n",ste_acc)
print("mean precision\n",prec_final)
print("ste of precision\n",ste_prec)
print("mean recall\n",rec)
print("ste of recall\n",ste_rec)
print("mean tnr/specificity\n",tnrate)
print("ste of tnr/specificity\n",ste_tnrate)

mean accuracy
 [[0.764 0.599 0.709 0.617]
 [0.541 0.644 0.497 0.507]
 [0.4   0.486 0.489 0.647]
 [0.54  0.667 0.65  0.581]
 [0.625 0.646 0.685 0.553]
 [0.82  0.712 0.763 0.566]
 [0.735 0.555 0.64  0.514]
 [0.788 0.706 0.858 0.608]
 [0.73  0.677 0.855 0.577]
 [0.762 0.689 0.853 0.574]]
ste of accuracy
 [[0.012 0.013 0.014 0.02 ]
 [0.017 0.011 0.002 0.016]
 [0.01  0.017 0.007 0.017]
 [0.017 0.015 0.016 0.016]
 [0.015 0.014 0.017 0.018]
 [0.01  0.009 0.014 0.019]
 [0.015 0.013 0.016 0.016]
 [0.014 0.012 0.01  0.016]
 [0.015 0.012 0.009 0.017]
 [0.014 0.01  0.009 0.016]]
mean precision
 [[0.498 0.343 0.424 0.396]
 [0.357 0.373 0.003 0.28 ]
 [0.057 0.257 0.039 0.527]
 [0.319 0.391 0.448 0.413]
 [0.446 0.378 0.525 0.299]
 [0.578 0.42  0.56  0.326]
 [0.522 0.311 0.425 0.291]
 [0.552 0.422 0.64  0.445]
 [0.516 0.391 0.636 0.403]
 [0.517 0.398 0.632 0.395]]
ste of precision
 [[0.016 0.01  0.019 0.028]
 [0.03  0.009 0.003 0.027]
 [0.014 0.015 0.015 0.035]
 [0.026 0.013 0.022 0.034]
 [0.025 0.012

In [4]:
#GPC(classifier index 2) data set 9(index 8), 1st repetition
predictions[2,8,:,1]

array([ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.])

In [5]:
#GPC(classifier index 2) data set 1(index 0), 1st repetition
predictions[2,0,:,1]

array([ 1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  0.])